In [14]:
import pandas as pd 
import xgboost as xgb
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [4]:
data = pd.read_csv('healthcare-dataset-stroke-data.csv')
stroke_data = data.drop(columns=['id']).dropna()

stroke_data

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5104,Female,13.0,0,0,No,children,Rural,103.08,18.6,Unknown,0
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [23]:
stroke_data.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [24]:
string_data = pd.DataFrame({'work_type': stroke_data['work_type'], 'smoking_status': stroke_data['smoking_status']})

encoder = OneHotEncoder(handle_unknown='ignore')
num_data = encoder.fit_transform(string_data[['work_type']]).toarray()


stroke_data['ever_married'] = (stroke_data['ever_married'] == 'yes').astype(int)
stroke_data['gender'] = (stroke_data['gender'] == 'Male').astype(int)
stroke_data['Residence_type'] = (stroke_data['Residence_type'] == 'Urban').astype(int)

stroke_data.update(num_data)

In [32]:
stroke_model = xgb.XGBClassifier(
                                # operation='binary:logistic',
                                n_estimators=100,
                                max_depth=5,
                                learning_rate=0.1,
                                random_state=10,
                                booster='gbtree'
                            )

x = stroke_data.drop(columns=['stroke', 'smoking_status', 'work_type'])
y = stroke_data['stroke']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

stroke_model.fit(x_train, y_train)
# stroke_model.fit(x_train._get_numeric_data(), np.ravel(y_train, order='C'))

stroke_prediction = stroke_model.predict(x_test)
model_accuracy = stroke_model.score(x_test, y_test)

model_accuracy

0.9576547231270358